En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [ ]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [ ]:
import cProfile
import pstats
import io
from memory_profiler import profile
import subprocess
import time
import os
from typing import Callable, Any, Dict, List
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class PerformanceAnalyzer:
    def __init__(self, output_dir: str = "performance_results"):
        """
        Inicializa el analizador de rendimiento.
        
        Args:
            output_dir: Directorio donde se guardarán los resultados
        """
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
    
    def measure_memory(self, func: Callable, *args, **kwargs) -> Dict[str, float]:
        """
        Mide el uso de memoria usando memory-profiler.
        
        Args:
            func: Función a medir
            args: Argumentos posicionales para la función
            kwargs: Argumentos nombrados para la función
            
        Returns:
            Dict con estadísticas de memoria
        """
        @profile
        def wrapped_func(*args, **kwargs):
            return func(*args, **kwargs)
        
        # Crear archivo de salida para memory-profiler
        memory_log = os.path.join(self.output_dir, f"{func.__name__}_memory.log")
        
        # Ejecutar con memory-profiler
        with open(memory_log, 'w') as log_file:
            wrapped_func(*args, **kwargs)
        
        # Analizar resultados
        memory_stats = self._parse_memory_profile(memory_log)
        return memory_stats
    
    def measure_time_profile(self, func: Callable, *args, **kwargs) -> Dict[str, float]:
        """
        Mide el tiempo de ejecución usando cProfile.
        
        Args:
            func: Función a medir
            args: Argumentos posicionales para la función
            kwargs: Argumentos nombrados para la función
            
        Returns:
            Dict con estadísticas de tiempo
        """
        profiler = cProfile.Profile()
        profiler.enable()
        
        result = func(*args, **kwargs)
        
        profiler.disable()
        
        # Guardar resultados detallados
        stats_file = os.path.join(self.output_dir, f"{func.__name__}_profile.stats")
        profiler.dump_stats(stats_file)
        
        # Analizar estadísticas
        s = io.StringIO()
        stats = pstats.Stats(profiler, stream=s).sort_stats('cumulative')
        stats.print_stats()
        
        return self._parse_cprofile_stats(s.getvalue())
    
    def measure_time_py_spy(self, func: Callable, *args, **kwargs) -> Dict[str, float]:
        """
        Mide el tiempo de ejecución usando py-spy.
        
        Args:
            func: Función a medir
            args: Argumentos posicionales para la función
            kwargs: Argumentos nombrados para la función
            
        Returns:
            Dict con estadísticas de tiempo
        """
        # Crear archivo para el perfil
        profile_file = os.path.join(self.output_dir, f"{func.__name__}_pyspy.svg")
        
        # Iniciar proceso con py-spy
        process = subprocess.Popen(
            ["py-spy", "record", "-o", profile_file, "--", "python", "-c", 
             f"from __main__ import {func.__name__}; {func.__name__}(*{args}, **{kwargs})"]
        )
        
        process.wait()
        
        return {
            "profile_file": profile_file
        }
    
    def _parse_memory_profile(self, log_file: str) -> Dict[str, float]:
        """
        Analiza el archivo de log de memory-profiler.
        """
        memory_stats = {
            "max_memory": 0,
            "min_memory": float('inf'),
            "avg_memory": 0
        }
        
        memories = []
        with open(log_file, 'r') as f:
            for line in f:
                if "MiB" in line:
                    try:
                        memory = float(line.split()[3])
                        memories.append(memory)
                    except (IndexError, ValueError):
                        continue
        
        if memories:
            memory_stats["max_memory"] = max(memories)
            memory_stats["min_memory"] = min(memories)
            memory_stats["avg_memory"] = sum(memories) / len(memories)
        
        return memory_stats
    
    def _parse_cprofile_stats(self, stats_string: str) -> Dict[str, float]:
        """
        Analiza la salida de cProfile.
        """
        lines = stats_string.split('\n')
        stats = {}
        
        for line in lines:
            if "function calls" in line:
                parts = line.split()
                stats["total_calls"] = int(parts[0])
            elif "seconds" in line:
                parts = line.split()
                stats["total_time"] = float(parts[0])
        
        return stats
    


In [ ]:
def run_performance_analysis(functions: List[Callable], test_data: str):
    """
    Ejecuta análisis de rendimiento completo para una lista de funciones.
    
    Args:
        functions: Lista de funciones a analizar
        test_data: Ruta al archivo de datos de prueba
    """
    analyzer = PerformanceAnalyzer()
    results = {}
    
    for func in functions:
        print(f"Analizando {func.__name__}...")
        
        # Medir memoria
        memory_stats = analyzer.measure_memory(func, test_data)
        
        # Medir tiempo con cProfile
        time_stats = analyzer.measure_time_profile(func, test_data)
        
        # Medir tiempo con py-spy
        pyspy_stats = analyzer.measure_time_py_spy(func, test_data)
        
        # Combinar resultados
        results[func.__name__] = {
            **memory_stats,
            **time_stats,
            **pyspy_stats
        }

    
    return results

In [ ]:
if __name__ == "__main__":
    functions_to_test = [
        q1_time, q1_memory,
        q2_time, q2_memory,
        q3_time, q3_memory
    ]
    
    results = run_performance_analysis(
        functions_to_test,
        file_path
    )
    
    # Imprimir resultados
    for func_name, stats in results.items():
        print(f"\nResultados para {func_name}:")
        print(f"Memoria máxima: {stats['max_memory']:.2f} MiB")
        print(f"Tiempo total: {stats['total_time']:.2f} segundos")
        print(f"Perfil detallado: {stats['profile_file']}")